# Sample project: stock prediction and trading

_Goal: to predict the price of a stock in the near future._



## Setup

Configure logging, import config, and connect to database.

In [1]:
%matplotlib inline

import os
import time
import datetime
import logging

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import config
from data.database import Database
from data.api_manager import API_Manager
from utils.descriptive_stats import weighted_mean

logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%H:%M:%S')
logger = logging.getLogger()
logger.setLevel(logging.INFO)

db = Database(config.database)
api = API_Manager(config.api_key)


## Fetch trades from API

Fetch all trades for ticker within date range and store them in the database.

In [2]:
ticker = 'MSFT'
date_from = '2015-01-01'
date_to = '2020-12-31'

ticker_details = db.get_ticker_details(ticker)
if ticker_details is None:
    ticker_details = db.store_ticker_details(api.get_ticker_details(ticker))

dates_with_trades = db.get_open_dates(ticker_details['exchange'], date_from, date_to)
dates_already_stored = db.get_stored_dates('trades', ticker)
dates_to_fetch = [d for d in dates_with_trades if d not in dates_already_stored]

logging.info(f'Fetching {len(dates_to_fetch)} days of {ticker} trades.')
for date in dates_to_fetch:
    
    # Download raw trades.
    time_before_fetch = time.time()
    trades = api.get_daily_trades(ticker, date)
    
    # Convert data to dataframe.
    time_before_process = time.time()
    trades = pd.DataFrame(trades)[['t', 'p', 's']]
    trades = trades.rename(columns={'t': 'timestamp', 'p': 'price', 's': 'volume'})

    # Store raw trades and bars in database.
    time_before_store = time.time()
    db.store_trades(ticker, date, trades)
    
    time_to_fetch = int(round(time_before_process - time_before_fetch))
    time_to_process = int(round(time_before_store - time_before_process))
    time_to_store = int(round(time.time() - time_before_store))
    logging.info(
        f'{ticker} {date} - '
        f'fetch time: {time_to_fetch}s, '
        f'process time: {time_to_process}s, '
        f'store time: {time_to_store}s'
    )

23:36:49 Fetching 0 days of MSFT trades.


## Feature engineering

For every `n` seconds during open hours, calculate target price and features. For now, `n` is set to `1`. Features are stored back in the database as they can take a while to calculate. 

Feature ideas:

- Summary stats of previous prices and volumes (https://alphascientist.com/feature_engineering.html).
- Trading markers (https://blog.roboforex.com/blog/2020/01/10/creating-a-trading-strategies-based-on-the-mean-reversion-and-momentum/).
- Stats on quotes.
- Relevant news articles.
- Mentions on social networks.


In [50]:
def stats_per_second(trades):
    """ Calculate descriptive stats for trades for every second.
    
    Stats are calculated in the second _before_ the time listed to prevent
    future data leakage.
    
    """
    trades_grouped = trades.groupby(
        pd.Grouper(key='time', freq='1S')
    )
    
    price_stats = trades_grouped['price'].agg(['count', 'mean', 'median', 'min', 'max', 'std'])
#     price_bars_weighted = trades_grouped.apply(get_weighted_bars)
    
    volume_stats = trades_grouped['volume'].agg(['sum', 'mean', 'median', 'min', 'max', 'std'])
    
    

ticker = 'MSFT'
exchange = db.get_ticker_details(ticker)['exchange']

stored_dates = sorted(db.get_stored_dates('trades', ticker))
open_hours = db.get_open_hours(stored_dates, exchange)

for date in [stored_dates[4], stored_dates[100]]:
    
    trades = db.get_trades(ticker, date)
    print(trades.iloc[0]['time'], trades.iloc[-1]['time'])
    
    open_time, close_time = open_hours[date]
    price_index = pd.date_range(
        datetime.datetime.combine(date, open_time),
        datetime.datetime.combine(date, close_time),
        freq='1S',
        closed='left'
    )
    period_grouper = pd.Grouper(key='time', freq='1S')
    
    prices = pd.DataFrame(index=price_index).merge(
        trades.groupby(period_grouper)['price'].median(),
        how='left',
        left_index=True,
        right_index=True
    )
#     prices = trades.groupby(period_grouper)['price'].median()
#     # Propegate last valid observation forward.
#     prices = prices.fillna(method='ffill')

prices.head()

2015-01-08 04:00:00.106000 2015-01-08 19:59:00.306000
2015-05-28 04:20:10.281000 2015-05-28 19:20:17.191000


,price
2015-05-28 09:00:00,NaN
2015-05-28 09:00:01,NaN
2015-05-28 09:00:02,NaN
2015-05-28 09:00:03,NaN
2015-05-28 09:00:04,NaN


,price
2015-05-28 11:46:40,47.55
2015-05-28 11:46:41,NaN
2015-05-28 11:46:42,47.55
2015-05-28 11:46:43,NaN
2015-05-28 11:46:44,NaN
...,...
2015-05-28 17:59:55,NaN
2015-05-28 17:59:56,NaN
2015-05-28 17:59:57,NaN
2015-05-28 17:59:58,NaN


### Target variable
The future price to predict. The variable classified into either `buy`, `don't sell`, or `sell` depending on the relative increase from the current price to the price in `m` seconds. For now, `m` is set to `5`. 

In [ ]:
ticker = 'MSFT'
exchange = db.get_ticker_details(ticker)['exchange']

frequency = '1S' # 1 second

stored_dates = sorted(db.get_stored_dates('trades', ticker))


trades

for date in stored_dates[::-1]:
    
    # Determine open hours (by Robinhood/Alpaca)

    trades = pd.DataFrame(
        db.get_trades(ticker, date), 
        columns=['timestamp', 'price', 'volume']
    )
    trades['time'] = pd.to_datetime(trades['timestamp']) - pd.DateOffset(hours=5) # America/New Yow
    
    price_index = pd.date_range(
        datetime.datetime.combine(date, start_time),
        datetime.datetime.combine(date, close_time),
        freq=frequency,
        closed='left'
    )
    
    prices = pd.DataFrame(index=price_index).merge(
        trades.groupby(pd.Grouper(key='time', freq=frequency))['price'].median(),
        how='left',
        left_index=True,
        right_index=True
    )
    
    # Propegate last valid observation forward.
    prices = prices.fillna(method='ffill')

    break
    
prices

description = 'Price'
db.store_feature(ticker, 'target', prices, description)

### Check features for outliers

### Check feature importance

## Hyperparameter tuning

- Feature time window
- Target variable future time